In [1]:
import glob
import cv2
import numpy as np
import os
import csv
import pandas as pd
from random import shuffle
from tqdm import tqdm
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [2]:
labels = {
    0: 'Basketball',
    1: 'Football',
    2: 'Rowing',
    3: 'Swimming',
    4: 'Tennis',
    5: 'Yoga', }
TRAIN_DIR = 'Train'
TEST_DIR = 'Test'
IMG_SIZE = 150
LR = 0.001
MODEL_NAME = 'Sport-cnn'
# -----------------------------------------#
def create_label(image_name):
    """ Create a one-hot encoded vector from image name """
    word_label = image_name.split('_')[0]
    if word_label == 'Basketball':
        return np.array([1, 0, 0, 0, 0, 0])
    elif word_label == 'Football':
        return np.array([0, 1, 0, 0, 0, 0])
    elif word_label == 'Rowing':
        return np.array([0, 0, 1, 0, 0, 0])
    elif word_label == 'Swimming':
        return np.array([0, 0, 0, 1, 0, 0])
    elif word_label == 'Tennis':
        return np.array([0, 0, 0, 0, 1, 0])
    elif word_label == 'Yoga':
        return np.array([0, 0, 0, 0, 0, 1])
#     else:
#         print(word_label,"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

In [3]:
def create_data(train_test):
    data = []
    img_count = 0
    folder = '/kaggle/input' +'/' + os.listdir('/kaggle/input')[0]
    for folder_type in os.listdir(folder):
        directory = folder+'/'+folder_type
#         print(directory)
        if folder_type.lower() != train_test.lower():
            print("skip",folder_type)
            continue
        print()
        for img_name in os.listdir(directory):
#             print("mame",img_name,"directory",directory)
            img_dir = directory+'/'+img_name
#             print(img_count,img_dir,)
            img_count += 1
            img_data = cv2.imread(img_dir)
            img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
            if train_test.lower() == 'train':
                data.append([np.array(img_data), create_label(img_name)])
            else :
                data.append(np.array(img_data))

    shuffle(data)
    print('number of images',img_count)
    return data,img_count


data,count = create_data("train")
print(np.array(data).shape)

skip Test

number of images 1681
(1681, 2)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
train_data, test_data = train_test_split(data, test_size=0.01, shuffle=False)
train = np.array(train_data)
test = test_data

# print((train).shape)
# [print(np.array(i[0]).shape) for i in train]
# [print(np.array(i[1]).shape) for i in train]

X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE,3)
y_train = [i[1] for i in train]

X_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE, IMG_SIZE,3)
y_test = [i[1] for i in test]

print('train',X_train.shape,X_test.shape,len(y_train),len(y_test))


train (1344, 150, 150, 3) (337, 150, 150, 3) 1344 337


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [5]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X


In [6]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [7]:
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    print(X.shape)
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [8]:
shape = X_train.shape
print(shape[1:])
base_model = ResNet50(shape[1:])

(150, 150, 3)
(None, 156, 156, 3)


2022-12-24 19:49:45.520304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 19:49:45.528177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 19:49:45.528865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 19:49:45.529934: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 6,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)


In [11]:
!wget "https://drive.google.com/uc?export=download&id=1n7rGZsAvwr6Le-IEHWBgA7ihz3pZA1DE"

--2022-12-24 19:49:48--  https://drive.google.com/uc?export=download&id=1n7rGZsAvwr6Le-IEHWBgA7ihz3pZA1DE
Resolving drive.google.com (drive.google.com)... 173.194.217.113, 173.194.217.138, 173.194.217.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pffjqksqokca36pi93fgfugv0ku90sm6/1671911325000/07448022858603959601/*/1n7rGZsAvwr6Le-IEHWBgA7ihz3pZA1DE?e=download&uuid=816a6bcf-3d6b-4d11-af91-d4c16b27c7c4 [following]
--2022-12-24 19:49:52--  https://doc-10-4k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pffjqksqokca36pi93fgfugv0ku90sm6/1671911325000/07448022858603959601/*/1n7rGZsAvwr6Le-IEHWBgA7ihz3pZA1DE?e=download&uuid=816a6bcf-3d6b-4d11-af91-d4c16b27c7c4
Resolving doc-10-4k-docs.googleusercontent.com (doc-10-4k-docs.googleusercontent.com)... 173.194.218.132, 

In [12]:
!mv "/kaggle/working/uc?export=download&id=1n7rGZsAvwr6Le-IEHWBgA7ihz3pZA1DE" "/kaggle/working/resnet50_weights.h5"

In [13]:
base_model.load_weights("/kaggle/working/resnet50_weights.h5")


In [14]:
model = Model(inputs=base_model.input, outputs=headModel)

In [15]:
y_train = np.array(y_train)
X_train = np.array(X_train)
print(type(y_train),type(X_train))
print(y_train.shape,X_train.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1344, 6) (1344, 150, 150, 3)


In [16]:
y_t = y_train.argmax(axis=1)

In [23]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01,nesterov=True,momentum=0.9), 
              loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [24]:
print(X_train.shape, y_train.shape)
history = model.fit(X_train,y_train,validation_split=0.2,batch_size=32,epochs=70)

(1344, 150, 150, 3) (1344, 6)
Epoch 1/70
34/34 [==============================] - 9s 134ms/step - loss: 9.5908e-05 - accuracy: 1.0000 - val_loss: 0.1926 - val_accuracy: 0.9368
Epoch 2/70
34/34 [==============================] - 4s 105ms/step - loss: 6.9476e-05 - accuracy: 1.0000 - val_loss: 0.1933 - val_accuracy: 0.9331
Epoch 3/70
34/34 [==============================] - 4s 106ms/step - loss: 8.1909e-05 - accuracy: 1.0000 - val_loss: 0.1991 - val_accuracy: 0.9331
Epoch 4/70
34/34 [==============================] - 4s 106ms/step - loss: 7.7190e-05 - accuracy: 1.0000 - val_loss: 0.2007 - val_accuracy: 0.9331
Epoch 5/70
34/34 [==============================] - 4s 107ms/step - loss: 8.1150e-05 - accuracy: 1.0000 - val_loss: 0.1971 - val_accuracy: 0.9294
Epoch 6/70
34/34 [==============================] - 4s 105ms/step - loss: 5.3119e-05 - accuracy: 1.0000 - val_loss: 0.1968 - val_accuracy: 0.9331
Epoch 7/70
34/34 [==============================] - 4s 106ms/step - loss: 5.3437e-05 - accurac

KeyboardInterrupt: 

In [21]:
pred_csv_data = []
x=[]
TEST_DIR = '/kaggle/input/nn23-sports-image-classification/Test'
for img in tqdm(os.listdir(TEST_DIR)):
    prediction = []
    path = os.path.join(TEST_DIR, img)
    img_ = cv2.imread(path)
    test_img = cv2.resize(img_, (IMG_SIZE, IMG_SIZE))
    test_img = test_img.reshape(IMG_SIZE, IMG_SIZE, 3)
    x.append([test_img])
    
x = np.array(x)
x = x.reshape(-1,IMG_SIZE, IMG_SIZE, 3)
print(x.shape)
prediction = model.predict(x)
i=0
for img in tqdm(os.listdir(TEST_DIR)):
    index = prediction[i].argmax()
#     print(index)
    pred_csv_data.append([img,index])
    i+=1
#index = prediction.argmax(axis=1)
#pred_csv_data.append([img,index])

pred_csv_header = ['image_name', 'label']
with open('test2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(pred_csv_header)
    # Use writerows() not writerow()
    writer.writerows(pred_csv_data)


100%|██████████| 688/688 [00:05<00:00, 119.41it/s]


(688, 150, 150, 3)


100%|██████████| 688/688 [00:00<00:00, 336287.28it/s]


In [20]:
pred_csv_header = ['image_name', 'label']
with open('ResNet.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(pred_csv_header)
    writer.writerows(result)

NameError: name 'result' is not defined